In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from shallowDict import SubjectDicionaryFCNet

def test_subject_specificity(batch_subject_ids, target_subject_ids, n_chans=22, n_times=1001, n_outputs=4):
    # Initialize the model
    model = SubjectDicionaryFCNet(n_chans=n_chans, n_outputs=n_outputs)
    
    # Set all weights and biases of fc_layers to ones for consistency
    for fc_layer in model.fc_layers.values():
        fc_layer.weight.data.fill_(1.0)
        fc_layer.bias.data.fill_(1.0)
    
    # Set requires_grad appropriately
    for param in model.parameters():
        param.requires_grad = False
    for fc_layer in model.fc_layers.values():
        for param in fc_layer.parameters():
            param.requires_grad = True
    
    # Copy initial weights and biases
    initial_weights = {k: v.weight.data.clone() for k, v in model.fc_layers.items()}
    initial_biases = {k: v.bias.data.clone() for k, v in model.fc_layers.items()}
    
    batch_size = len(batch_subject_ids)
    
    # Create batch data
    batch = torch.zeros(batch_size, n_chans, n_times)
    
    # Initialize target
    target = torch.zeros(batch_size, n_outputs)
    
    # Set batch data and target
    for i, subject_id in enumerate(batch_subject_ids):
        if subject_id in target_subject_ids:
            batch[i] = 1.0  # Or any meaningful data
            target[i] = torch.tensor([1.0, 2.0, 3.0, 4.0])  # Non-zero target
        else:
            batch[i] = 0.0  # Or any other data
            target[i] = 0.0  # Zero target
        
        # Correctly encode the subject ID
        batch[i, :, -1] = subject_id * 1000000

    # Define a loss function
    criterion = nn.MSELoss()
    
    # Define an optimizer
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
    
    # Training step
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(batch)

    loss = criterion(output, target)
    # Print loss value
    print(f"Loss: {loss.item()}")
    
    # Backward pass
    loss.backward()
    
    # Verify gradients before optimizer step
    print("\nGradients before optimizer step:")
    for subject_id_str, fc_layer in model.fc_layers.items():
        grad_norm = fc_layer.weight.grad.norm().item() if fc_layer.weight.grad is not None else 0
        print(f"Gradient norm for {subject_id_str}: {grad_norm}")
    
    # Update weights
    optimizer.step()
    
    # Check which fc_layers have had their weights updated
    for subject_id_str, fc_layer in model.fc_layers.items():
        weights_changed = (fc_layer.weight.data != initial_weights[subject_id_str]).float().sum().item()
        biases_changed = (fc_layer.bias.data != initial_biases[subject_id_str]).float().sum().item()
        print(f"\nWeights changed for {subject_id_str}: {weights_changed}")
        print(f"Biases changed for {subject_id_str}: {biases_changed}")
    
    # Assertions to verify subject specificity
    for subject_id_str, fc_layer in model.fc_layers.items():
        subject_id_int = int(subject_id_str.split('_')[1])  # Convert 'subject_1' to 1
        weights_changed = (fc_layer.weight.data != initial_weights[subject_id_str]).float().sum().item()
        biases_changed = (fc_layer.bias.data != initial_biases[subject_id_str]).float().sum().item()
        if subject_id_int in target_subject_ids:
            assert weights_changed > 0, f"Weights for {subject_id_str} did not change!"
            assert biases_changed > 0, f"Biases for {subject_id_str} did not change!"
        else:
            assert weights_changed == 0, f"Weights for {subject_id_str} have changed!"
            assert biases_changed == 0, f"Biases for {subject_id_str} have changed!"
    
    print("\nTest passed: Only the weights and biases corresponding to the specified subjects have changed.")


# Test with one specific subject
batch_subject_ids = [3]
target_subject_ids = [3]
test_subject_specificity(batch_subject_ids, target_subject_ids)

# Test with two specific subjects
# batch_subject_ids = [1, 2, 3, 4, 5]
# target_subject_ids = [2,3]
# test_subject_specificity(batch_subject_ids, target_subject_ids)

# # Test with all but 1 ssubject
# batch_subject_ids = [1, 2, 3, 4, 5]
# target_subject_ids = [1,2,3,4]
# test_subject_specificity(batch_subject_ids, target_subject_ids)

Loss: 3.5

Gradients before optimizer step:
Gradient norm for subject_1: 0
Gradient norm for subject_2: 0
Gradient norm for subject_3: 0.0
Gradient norm for subject_4: 0
Gradient norm for subject_5: 0
Gradient norm for subject_6: 0
Gradient norm for subject_7: 0
Gradient norm for subject_8: 0
Gradient norm for subject_9: 0

Weights changed for subject_1: 0.0
Biases changed for subject_1: 0.0

Weights changed for subject_2: 0.0
Biases changed for subject_2: 0.0

Weights changed for subject_3: 0.0
Biases changed for subject_3: 3.0

Weights changed for subject_4: 0.0
Biases changed for subject_4: 0.0

Weights changed for subject_5: 0.0
Biases changed for subject_5: 0.0

Weights changed for subject_6: 0.0
Biases changed for subject_6: 0.0

Weights changed for subject_7: 0.0
Biases changed for subject_7: 0.0

Weights changed for subject_8: 0.0
Biases changed for subject_8: 0.0

Weights changed for subject_9: 0.0
Biases changed for subject_9: 0.0


AssertionError: Weights for subject_3 did not change!

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from shallowDict import ShallowPrivateCollapsedDictNetSlow

def test_subject_specificity(batch_subject_ids, target_subject_ids, n_chans=22, n_times=1001, n_outputs=4):
    # Initialize the model
    model = ShallowPrivateCollapsedDictNetSlow(n_chans=n_chans, n_outputs=n_outputs)
    
    # Set all weights and biases of fc_layers to ones for consistency
    for spatio_temporal_layer in model.spatio_temporal_layers.values():
        spatio_temporal_layer.weight.data.fill_(1.0)
        spatio_temporal_layer.bias.data.fill_(1.0)
    
    # Set requires_grad appropriately
    for param in model.parameters():
        param.requires_grad = False
    for spatio_temporal_layer in model.spatio_temporal_layers.values():
        for param in spatio_temporal_layer.parameters():
            param.requires_grad = True
    #for param in model.batch_norm.parameters():
    #    param.requires_grad = True
    
    # Copy initial weights and biases
    initial_weights = {k: v.weight.data.clone() for k, v in model.spatio_temporal_layers.items()}
    initial_biases = {k: v.bias.data.clone() for k, v in model.spatio_temporal_layers.items()}
    
    batch_size = len(batch_subject_ids)
    
    # Create batch data
    batch = torch.zeros(batch_size, n_chans, n_times)
    
    # Initialize target
    target = torch.zeros(batch_size, n_outputs)
    
    # Set batch data and target
    for i, subject_id in enumerate(batch_subject_ids):
        if subject_id in target_subject_ids:
            batch[i] = 1.0  # Or any meaningful data
            target[i] = torch.tensor([1.0, 2.0, 3.0, 4.0])  # Non-zero target
        else:
            batch[i] = 0.0  # Or any other data
            target[i] = 0.0  # Zero target
        
        # Correctly encode the subject ID
        batch[i, :, -1] = subject_id * 1000000

    # Define a loss function
    criterion = nn.MSELoss()
    
    # Define an optimizer
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
    
    # Training step
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(batch)
    

    # Compute loss only for target subjects

    loss = criterion(output, target)

    
    # Print loss value
    print(f"Loss: {loss.item()}")
    
    # Backward pass
    loss.backward()
    
    # Verify gradients before optimizer step
    print("\nGradients before optimizer step:")
    for subject_id_str, spatio_temporal_layer in model.spatio_temporal_layers.items():
        grad_norm = spatio_temporal_layer.weight.grad.norm().item() if spatio_temporal_layer.weight.grad is not None else 0
        print(f"Gradient norm for {subject_id_str}: {grad_norm}")
    
    # Update weights
    optimizer.step()
    
    # Check which fc_layers have had their weights updated
    for subject_id_str, spatio_temporal_layer in model.spatio_temporal_layers.items():
        weights_changed = (spatio_temporal_layer.weight.data != initial_weights[subject_id_str]).float().sum().item()
        biases_changed = (spatio_temporal_layer.bias.data != initial_biases[subject_id_str]).float().sum().item()
        print(f"\nWeights changed for {subject_id_str}: {weights_changed}")
        print(f"Biases changed for {subject_id_str}: {biases_changed}")
    
    # Assertions to verify subject specificity
    for subject_id_str, spatio_temporal_layer in model.spatio_temporal_layers.items():
        subject_id_int = int(subject_id_str.split('_')[1])  # Convert 'subject_1' to 1
        weights_changed = (spatio_temporal_layer.weight.data != initial_weights[subject_id_str]).float().sum().item()
        biases_changed = (spatio_temporal_layer.bias.data != initial_biases[subject_id_str]).float().sum().item()
        if subject_id_int in target_subject_ids:
            assert weights_changed > 0, f"Weights for {subject_id_str} did not change!"
            assert biases_changed > 0, f"Biases for {subject_id_str} did not change!"
        else:
            assert weights_changed == 0, f"Weights for {subject_id_str} have changed!"
            assert biases_changed == 0, f"Biases for {subject_id_str} have changed!"
    
    print("\nTest passed: Only the weights and biases corresponding to the specified subjects have changed.")


# Test with one specific subject
batch_subject_ids = [3]
target_subject_ids = [3]
test_subject_specificity(batch_subject_ids, target_subject_ids)

# Test with two specific subjects
batch_subject_ids = [2]
target_subject_ids = [2]
test_subject_specificity(batch_subject_ids, target_subject_ids)

# # Test with all but 1 ssubject
# batch_subject_ids = [1, 2, 3, 4, 5]
# target_subject_ids = [1,2,3,4]
# test_subject_specificity(batch_subject_ids, target_subject_ids)

Loss: 7.404048919677734

Gradients before optimizer step:
Gradient norm for subject_1: 0
Gradient norm for subject_2: 0
Gradient norm for subject_3: 0.024984247982501984
Gradient norm for subject_4: 0
Gradient norm for subject_5: 0
Gradient norm for subject_6: 0
Gradient norm for subject_7: 0
Gradient norm for subject_8: 0
Gradient norm for subject_9: 0

Weights changed for subject_1: 0.0
Biases changed for subject_1: 0.0

Weights changed for subject_2: 0.0
Biases changed for subject_2: 0.0

Weights changed for subject_3: 20906.0
Biases changed for subject_3: 35.0

Weights changed for subject_4: 0.0
Biases changed for subject_4: 0.0

Weights changed for subject_5: 0.0
Biases changed for subject_5: 0.0

Weights changed for subject_6: 0.0
Biases changed for subject_6: 0.0

Weights changed for subject_7: 0.0
Biases changed for subject_7: 0.0

Weights changed for subject_8: 0.0
Biases changed for subject_8: 0.0

Weights changed for subject_9: 0.0
Biases changed for subject_9: 0.0

Test pas

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
from shallowDict import ShallowPrivateTemporalDictNetSlow

def test_subject_specificity(batch_subject_ids, target_subject_ids, n_chans=22, n_times=1001, n_outputs=4):
    # Initialize the model
    model = ShallowPrivateTemporalDictNetSlow(n_chans=n_chans, n_outputs=n_outputs)
    
    # Set all weights and biases of fc_layers to ones for consistency
    for temporal_layer in model.temporal_layers.values():
        temporal_layer.weight.data.fill_(1.0)
        temporal_layer.bias.data.fill_(1.0)
    
    # Set requires_grad appropriately
    for param in model.parameters():
        param.requires_grad = False
    for temporal_layer in model.temporal_layers.values():
        for param in temporal_layer.parameters():
            param.requires_grad = True
    #for param in model.batch_norm.parameters():
    #    param.requires_grad = True
    
    # Copy initial weights and biases
    initial_weights = {k: v.weight.data.clone() for k, v in model.temporal_layers.items()}
    initial_biases = {k: v.bias.data.clone() for k, v in model.temporal_layers.items()}
    
    batch_size = len(batch_subject_ids)
    
    # Create batch data
    batch = torch.zeros(batch_size, n_chans, n_times)
    
    # Initialize target
    target = torch.zeros(batch_size, n_outputs)
    
    # Set batch data and target
    for i, subject_id in enumerate(batch_subject_ids):
        if subject_id in target_subject_ids:
            batch[i] = 1.0  # Or any meaningful data
            target[i] = torch.tensor([1.0, 2.0, 3.0, 4.0])  # Non-zero target
        else:
            batch[i] = 0.0  # Or any other data
            target[i] = 0.0  # Zero target
        
        # Correctly encode the subject ID
        batch[i, :, -1] = subject_id * 1000000

    # Define a loss function
    criterion = nn.MSELoss()
    
    # Define an optimizer
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
    
    # Training step
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(batch)
    

    loss = criterion(output, target)
    
    # Print loss value
    print(f"Loss: {loss.item()}")
    
    # Backward pass
    loss.backward()
    
    # Verify gradients before optimizer step
    print("\nGradients before optimizer step:")
    for subject_id_str, temporal_layer in model.temporal_layers.items():
        grad_norm = temporal_layer.weight.grad.norm().item() if temporal_layer.weight.grad is not None else 0
        print(f"Gradient norm for {subject_id_str}: {grad_norm}")
    
    # Update weights
    optimizer.step()
    
    # Check which fc_layers have had their weights updated
    for subject_id_str, temporal_layer in model.temporal_layers.items():
        weights_changed = (temporal_layer.weight.data != initial_weights[subject_id_str]).float().sum().item()
        biases_changed = (temporal_layer.bias.data != initial_biases[subject_id_str]).float().sum().item()
        print(f"\nWeights changed for {subject_id_str}: {weights_changed}")
        print(f"Biases changed for {subject_id_str}: {biases_changed}")
    
    # Assertions to verify subject specificity
    for subject_id_str, temporal_layer in model.temporal_layers.items():
        subject_id_int = int(subject_id_str.split('_')[1])  # Convert 'subject_1' to 1
        weights_changed = (temporal_layer.weight.data != initial_weights[subject_id_str]).float().sum().item()
        biases_changed = (temporal_layer.bias.data != initial_biases[subject_id_str]).float().sum().item()
        if subject_id_int in target_subject_ids:
            assert weights_changed > 0, f"Weights for {subject_id_str} did not change!"
            assert biases_changed > 0, f"Biases for {subject_id_str} did not change!"
        else:
            assert weights_changed == 0, f"Weights for {subject_id_str} have changed!"
            assert biases_changed == 0, f"Biases for {subject_id_str} have changed!"
    
    print("\nTest passed: Only the weights and biases corresponding to the specified subjects have changed.")


# Test with one specific subject
batch_subject_ids = [3]
target_subject_ids = [3]
test_subject_specificity(batch_subject_ids, target_subject_ids)

# Test with two specific subjects
batch_subject_ids = [2]
target_subject_ids = [2]
test_subject_specificity(batch_subject_ids, target_subject_ids)

# # Test with all but 1 ssubject
# batch_subject_ids = [1, 2, 3, 4, 5]
# target_subject_ids = [1,2,3,4]
# test_subject_specificity(batch_subject_ids, target_subject_ids)

Loss: 7.641894340515137

Gradients before optimizer step:
Gradient norm for subject_1: 0
Gradient norm for subject_2: 0
Gradient norm for subject_3: 0.002165168523788452
Gradient norm for subject_4: 0
Gradient norm for subject_5: 0
Gradient norm for subject_6: 0
Gradient norm for subject_7: 0
Gradient norm for subject_8: 0
Gradient norm for subject_9: 0

Weights changed for subject_1: 0.0
Biases changed for subject_1: 0.0

Weights changed for subject_2: 0.0
Biases changed for subject_2: 0.0

Weights changed for subject_3: 1000.0
Biases changed for subject_3: 37.0

Weights changed for subject_4: 0.0
Biases changed for subject_4: 0.0

Weights changed for subject_5: 0.0
Biases changed for subject_5: 0.0

Weights changed for subject_6: 0.0
Biases changed for subject_6: 0.0

Weights changed for subject_7: 0.0
Biases changed for subject_7: 0.0

Weights changed for subject_8: 0.0
Biases changed for subject_8: 0.0

Weights changed for subject_9: 0.0
Biases changed for subject_9: 0.0

Test pass

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from shallowDict import ShallowPrivateSpatialDictNetSlow

def test_subject_specificity(batch_subject_ids, target_subject_ids, n_chans=22, n_times=1001, n_outputs=4):
    # Initialize the model
    model = ShallowPrivateSpatialDictNetSlow(n_chans=n_chans, n_outputs=n_outputs)
    
    # Set all weights and biases of fc_layers to ones for consistency
    for spatial_layer in model.spatial_layers.values():
        spatial_layer.weight.data.fill_(1.0)
        spatial_layer.bias.data.fill_(1.0)
    
    # Set requires_grad appropriately
    for param in model.parameters():
        param.requires_grad = False
    for spatial_layer in model.spatial_layers.values():
        for param in spatial_layer.parameters():
            param.requires_grad = True
    #for param in model.batch_norm.parameters():
    #    param.requires_grad = True
    
    # Copy initial weights and biases
    initial_weights = {k: v.weight.data.clone() for k, v in model.spatial_layers.items()}
    initial_biases = {k: v.bias.data.clone() for k, v in model.spatial_layers.items()}
    
    batch_size = len(batch_subject_ids)
    
    # Create batch data
    batch = torch.zeros(batch_size, n_chans, n_times)
    
    # Initialize target
    target = torch.zeros(batch_size, n_outputs)
    
    # Set batch data and target
    for i, subject_id in enumerate(batch_subject_ids):
        if subject_id in target_subject_ids:
            batch[i] = 1.0  # Or any meaningful data
            target[i] = torch.tensor([1.0, 2.0, 3.0, 4.0])  # Non-zero target
        else:
            batch[i] = 0.0  # Or any other data
            target[i] = 0.0  # Zero target
        
        # Correctly encode the subject ID
        batch[i, :, -1] = subject_id * 1000000

    # Define a loss function
    criterion = nn.MSELoss()
    
    # Define an optimizer
    optimizer = optim.SGD(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)
    
    # Training step
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output = model(batch)
    
    # Compute loss only for target subjects

    loss = criterion(output, target)

    
    # Print loss value
    print(f"Loss: {loss.item()}")
    
    # Backward pass
    loss.backward()
    
    # Verify gradients before optimizer step
    print("\nGradients before optimizer step:")
    for subject_id_str, spatial_layer in model.spatial_layers.items():
        grad_norm = spatial_layer.weight.grad.norm().item() if spatial_layer.weight.grad is not None else 0
        print(f"Gradient norm for {subject_id_str}: {grad_norm}")
    
    # Update weights
    optimizer.step()
    
    # Check which fc_layers have had their weights updated
    for subject_id_str, spatial_layer in model.spatial_layers.items():
        weights_changed = (spatial_layer.weight.data != initial_weights[subject_id_str]).float().sum().item()
        biases_changed = (spatial_layer.bias.data != initial_biases[subject_id_str]).float().sum().item()
        print(f"\nWeights changed for {subject_id_str}: {weights_changed}")
        print(f"Biases changed for {subject_id_str}: {biases_changed}")
    
    # Assertions to verify subject specificity
    for subject_id_str, spatial_layer in model.spatial_layers.items():
        subject_id_int = int(subject_id_str.split('_')[1])  # Convert 'subject_1' to 1
        weights_changed = (spatial_layer.weight.data != initial_weights[subject_id_str]).float().sum().item()
        biases_changed = (spatial_layer.bias.data != initial_biases[subject_id_str]).float().sum().item()
        if subject_id_int in target_subject_ids:
            assert weights_changed > 0, f"Weights for {subject_id_str} did not change!"
            assert biases_changed > 0, f"Biases for {subject_id_str} did not change!"
        else:
            assert weights_changed == 0, f"Weights for {subject_id_str} have changed!"
            assert biases_changed == 0, f"Biases for {subject_id_str} have changed!"
    
    print("\nTest passed: Only the weights and biases corresponding to the specified subjects have changed.")


# Test with one specific subject
batch_subject_ids = [3]
target_subject_ids = [3]
test_subject_specificity(batch_subject_ids, target_subject_ids)

# Test with two specific subjects
batch_subject_ids = [2]
target_subject_ids = [2]
test_subject_specificity(batch_subject_ids, target_subject_ids)

# Test with all but 1 ssubject
# batch_subject_ids = [1, 2, 3, 4, 5]
# target_subject_ids = [1,2,3,4]
# test_subject_specificity(batch_subject_ids, target_subject_ids)

Loss: 7.432461261749268

Gradients before optimizer step:
Gradient norm for subject_1: 0
Gradient norm for subject_2: 0
Gradient norm for subject_3: 0.0324491485953331
Gradient norm for subject_4: 0
Gradient norm for subject_5: 0
Gradient norm for subject_6: 0
Gradient norm for subject_7: 0
Gradient norm for subject_8: 0
Gradient norm for subject_9: 0

Weights changed for subject_1: 0.0
Biases changed for subject_1: 0.0

Weights changed for subject_2: 0.0
Biases changed for subject_2: 0.0

Weights changed for subject_3: 31790.0
Biases changed for subject_3: 39.0

Weights changed for subject_4: 0.0
Biases changed for subject_4: 0.0

Weights changed for subject_5: 0.0
Biases changed for subject_5: 0.0

Weights changed for subject_6: 0.0
Biases changed for subject_6: 0.0

Weights changed for subject_7: 0.0
Biases changed for subject_7: 0.0

Weights changed for subject_8: 0.0
Biases changed for subject_8: 0.0

Weights changed for subject_9: 0.0
Biases changed for subject_9: 0.0

Test passe